# Лекция 6: Компьютерное зрение

## История развития

В 50–70-х годах XX века компьютерное зрение начинало зарождаться как междисциплинарная область, объединяющая исследования в области обработки изображений, анализа форм и искусственного интеллекта, в те времена, основанного, по большей части, на принятии решений.  

На ранних этапах активно применялись методы фильтрации, выделения краёв (операторы Собеля, Превитта, Канни), преобразование Хафа для обнаружения линий и геометрических фигур.

В 80–90-х годах развитие компьютерного зрения характеризовалось переходом от чисто геометрических методов к статистическим подходам, появились алгоритмы для выделения устойчивых точек интереса, распознавания объектов, основываясь на вручную сконструированных признаках.

Одним из переломных моментов стала идея использования нейронных сетей для обработки изображений:
- *Neocognitron (1979):* Первая иерархическую архитектура, имеющая схожесть с современными сверточными нейронными сетями (CNN). 
- *LeNet (1989):* Ян Лекун и др. разработали LeNet для распознавания рукописных цифр, показав эффективность принципа локальной фильтрации и пулинга.

С началом 2010-х годов благодаря доступности больших наборов данных (самый известный пример - ImageNet) и возрастанию вычислительных мощностей GPU и появления API для CUDA, было произведено возвращение CNN и увеличение числа слоев. Так появились:
- *AlexNet (2012):* Прорывная архитектура, показавшая существенное снижение ошибки классификации в соревновании на датасете ImageNet.
- *VGG:* Простая архитектуры со сверточными ядрами $ (3x3) $.
- *GoogLeNet/Inception:* Вложенные модули, позволяющие учитывать признаки разных масштабов.
- *ResNet:* Введение остаточных связей для борьбы с проблемой затухающих градиентов, что позволило строить очень глубокие сети.
- *YOLO:* Предоставление предсказывай граничных полей и вероятности классов по полным изображениям за одну оценку.

<img src="https://www.researchgate.net/publication/350085174/figure/fig3/AS:1022710452854786@1620844581128/Computer-Vision-Techniques-evolution-from-2012-to-2019-78-Alex-Net-78-Boosted-Cascade.png" alt="Описание изображения" width="800">

## Типы задач компьютерного зрения

Компьютерное зрение охватывает широкий спектр задач, каждая из которых требует специфических методов и подходов, прежде всего это касается разметки изображений. 

К основным задачам относятся:
1. *Классификация изображений* - определение категории или метки всего изображения.
2. *Детекция объектов* - Поиск и локализация объектов на изображении с указанием их координат.   
3. *Сегментация изображений*
   - *Семантическая сегментация* - классификация каждого пикселя изображения на основе принадлежности к определённому классу.
   - *Инстанс-сегментация* - обнаружение отдельных экземпляров объектов с учетом их границ.
4. *Распознавание объектов (лиц)* - детекция и идентификация объектов (по сути, совмещение задач классификации и детекции)
5. *Работа с пространственными данными (облаками точек)* - определение глубины изображения и построение трёхмерной модели.
6. *Анализ и обработка видео* - трекинг объектов, детектирование движений и распознавания действий.

<img src="https://www.researchgate.net/profile/Dae-Young-Kang/publication/346091812/figure/fig5/AS:979480482955270@1610537753983/Computer-vision-tasks-Adapted-from.png" alt="Описание изображения" width="800">

## Архитектуры

Изображения представляют собой двумерные (хотя чаще трёхмерные, с учетом цветовых каналов) массивы пикселей - это исходные признаки. Положение пикселей в пространстве, как и непосредственно сигнал в них, должны каким-то образом быть учтены и обобщены. Для этого необходимо извлечь из исходных данных признаки более высокого уровня - для этого и нужна именно глубокая сеть.

**Мотивировка:** 
- Поскольку соседние пиксели часто содержат схожую или взаимодополняющую информацию применение **операции свертки** кажется логичным. С помощью нее извлекаются локальных признаки вроде границ, текстур, углов.
- По мере продвижения по слоям сети, признаки комбинируются в более сложные и **высокоуровневые признаки** - формы, паттерны и даже образы.
- Необходимо обеспечить устойчивость к смещениям, масштабированию, поворотам, изменениям освещенности. Для этого используется операция **пулинга** и иные архитектурные решения, а также **аугментация** данных для обучения.

### Сверточные сети

Обозначим входное изображение как тензор:
$$
\mathbf{I} \in \mathbb{R}^{H \times W \times C},
$$
где $ H $ — высота, $ W $ — ширина, $ C $ — число каналов (например, 3 для RGB).

Для каждого сверточного слоя используются фильтры (ядра), которые представляют собой матрицы обучаемых параметров:
$$
\mathbf{K} \in \mathbb{R}^{k_h \times k_w \times C \times F},
$$
где:
- $k_h$ и $k_w$ — размеры фильтра по высоте и ширине,
- $C$ — число входных каналов (совпадает с числом каналов изображения или выходным числом предыдущего слоя),
- $F$ — количество фильтров.

Операция свертки вычисляется следующим образом. Пусть $ \mathbf{O} $ — выходной тензор, тогда для каждого фильтра $ f $ и позиции $(i, j)$ выход записывается как:
$$
O(i, j, f) = \sum_{c=1}^{C} \sum_{u=0}^{k_h - 1} \sum_{v=0}^{k_w - 1} K(u, v, c, f) \cdot I(i + u, j + v, c) + b(f).
$$

Для удобства описания вводятся следующие термины:
- *Stride* - шаг свертки $ s $ определяет, на сколько пикселей сдвигается фильтр вдоль осей $ H $ и $ W $.
- *Padding* - чтобы сохранить размерность или задать определенные граничные условия, к входному изображению добавляются нули по краям (zero-padding). Если используется паддинг $ p $, то размер выхода будет:
  $$
  H_{\text{out}} = \frac{H - k_h + 2p}{s} + 1, \quad
  W_{\text{out}} = \frac{W - k_w + 2p}{s} + 1.
  $$

После операции свертки на каждом элементе выходного тензора применяется нелинейная функция активации (обычно ReLU):
$$
Y(i,j,f) = \phi\Bigl(O(i,j,f)\Bigr) = \max(0, O(i,j,f)\Bigr).
$$

Пулинг-слои применяются для агрегации информации и снижения размерности. Наиболее распространены:
- *Max Pooling* - выбирается максимальное значение в заданном окне.
- *Average Pooling* - вычисляется среднее значение пикселей в окне.

Если окно имеет размер $ p \times p $ с шагом $ s_p $, то:
$$
P(i,j,f) = \max_{\substack{0 \leq u < p\\0 \leq v < p}} Y(i\cdot s_p + u, j\cdot s_p + v, f)
$$

<img src="https://saturncloud.io/images/blog/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way.webp" alt="Описание изображения" width="600">

<img src="https://contenthub-static.grammarly.com/blog/wp-content/uploads/2024/09/157421-grammarly-6162-blogvisuals-cnns-Image2-Op1-B_V1_new.png" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm


class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.dropout = nn.Dropout(0.25)

        self.fc1 = nn.Linear(256 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout(x)

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


def train(model, device, train_loader, optimizer, criterion, epoch):
    """
    Обучение модели на одной эпохе.
    """
    model.train()
    running_loss = 0.0

    for batch_idx, (data, target) in tqdm(enumerate(train_loader), 
                                          total=len(train_loader),
                                          desc=f'Epoch {epoch}'):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * data.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}. Loss: {epoch_loss:.4f}")
    return epoch_loss


def test(model, device, test_loader, criterion):
    """
    Тестирование модели на тестовой выборке.
    """
    model.eval()
    test_loss = 0.0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, "
          f"Accuracy: {correct}/{len(test_loader.dataset)} "
          f"({accuracy:.2f}%)")
    return test_loss, accuracy


def visualize_activations(model, device, image):
    """
    Визуализация активаций сверточных слоев для одного изображения.
    """
    activations = {}

    def get_activation(name):
        def hook(model, input, output):
            activations[name] = output.detach().cpu()
        return hook

    model.conv1.register_forward_hook(get_activation('conv1'))
    model.conv2.register_forward_hook(get_activation('conv2'))
    model.conv3.register_forward_hook(get_activation('conv3'))
    model.conv4.register_forward_hook(get_activation('conv4'))

    model.eval()
    with torch.no_grad():
        image = image.to(device)
        _ = model(image)

    for layer_name, activation in activations.items():
        num_channels = activation.shape[1]
        num_plots = min(num_channels, 6)
        fig, axes = plt.subplots(1, num_plots, figsize=(10, 3))
        fig.suptitle(f'Активности слоя {layer_name}', fontsize=16)
        for i in range(num_plots):
            ax = axes[i] if num_plots > 1 else axes
            feature_map = activation[0, i, :, :].numpy()
            ax.imshow(feature_map, cmap='viridis')
            ax.axis('off')
            ax.set_title(f'Канал {i}')
        plt.tight_layout()
        plt.show()


batch_size = 64
test_batch_size = 1000
epochs = 3
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, 
                               transform=transform)
subset_train = np.arange(10000)
train_dataset = Subset(train_dataset, subset_train)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, 
                              transform=transform)
subset_test = np.arange(1000)
test_dataset = Subset(test_dataset, subset_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, 
                         shuffle=False)

model = DeepCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(1, epochs + 1):
    train_loss = train(model, device, train_loader, 
                       optimizer, criterion, epoch)
    
    train_losses.append(train_loss)
    test_loss, accuracy = test(model, device, test_loader, criterion)
    test_losses.append(test_loss)
    test_accuracies.append(accuracy)

epochs_range = np.arange(1, epochs + 1)
fig, ax1 = plt.subplots(figsize=(8, 4))
ax1.plot(epochs_range, train_losses, 'o-', label='Train Loss')
ax1.plot(epochs_range, test_losses, 's-', label='Test Loss')
ax1.set_xlabel('Эпоха')
ax1.set_ylabel('Loss')
ax1.set_title('График сходимости и метрика распознавания')
ax1.legend(loc='upper left')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(epochs_range, test_accuracies, 'd--', color='red', 
         label='Test Accuracy')
ax2.set_ylabel('Accuracy (%)')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

sample_image, sample_label = test_dataset[0]
sample_image = sample_image.unsqueeze(0)
visualize_activations(model, device, sample_image)

### Трансформеры

Трансформерные модели, изначально разработанные для обработки последовательностей в NLP, адаптируются для анализа изображений. Ключевая идея состоит в разбиении изображения на патчи и последующей обработке их как последовательности токенов.

#### Vision Transformer (ViT)
##### Разбиение изображений на патчи
Пусть имеется изображение $ \mathbf{I} \in \mathbb{R}^{H \times W \times C} $. Его разбивают на $ N $ неперекрывающихся патчей размером $ P \times P $ (при условии, что $ H $ и $ W $ кратны $ P $). Каждый патч выпрямляется в вектор:
$$
\mathbf{x}_i \in \mathbb{R}^{P^2 \cdot C}, \quad i = 1, \dots, N,
$$
где $ N = \frac{H \times W}{P^2} $.

##### Линейное вложение и позиционные кодировки
Каждый выпрямленный патч отображается в пространство фиксированной размерности $ D $ с помощью линейного слоя:
$$
\mathbf{z}_i = \mathbf{E} \, \mathbf{x}_i,
$$
где $ \mathbf{E} \in \mathbb{R}^{D \times (P^2 \cdot C)} $ — матрица вложения. Для сохранения информации о порядке патчей добавляют позиционные кодировки:
$$
\mathbf{z}_i = \mathbf{z}_i + \mathbf{p}_i,
$$
где $ \mathbf{p}_i $ — позиционный эмбеддинг-вектор для $ i $-го патча.

##### Трансформер-энкодер
Последовательность эмбеддингов $\{ \mathbf{z}_1, \dots, \mathbf{z}_N \}$ подается на стандартный блок трансформера, состоящий из:

1. *Многошагового механизма самовнимания (Multi-Head Self-Attention, MHA):*
   Для входной матрицы $ \mathbf{Z} \in \mathbb{R}^{N \times D} $ вычисляются запросы (Query), ключи (Key) и значения (Value):
   $$
   \mathbf{Q} = \mathbf{Z} \mathbf{W}^Q, \quad \mathbf{K} = \mathbf{Z} \mathbf{W}^K, \quad \mathbf{V} = \mathbf{Z} \mathbf{W}^V,
   $$
   где $ \mathbf{W}^Q, \mathbf{W}^K, \mathbf{W}^V \in \mathbb{R}^{D \times d_k} $ — матрицы преобразования. Механизм самовнимания вычисляется по формуле:
   $$
   \text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \operatorname{softmax}\left(\frac{\mathbf{Q}\mathbf{K}^T}{\sqrt{d_k}}\right) \mathbf{V}.
   $$
   Множество таких heads, каждая со своим набором матриц $ \mathbf{W}^Q, \mathbf{W}^K, \mathbf{W}^V $ вычисляются параллельно, после чего их результаты объединяются.

2. *Feed-Forward Network:*
   Каждая позиция обрабатывается независимой двухслойной сетью (Multi-Layer Perceptron):
   $$
   \text{FFN}(\mathbf{z}) = \max(0, \mathbf{z} \mathbf{W}_1 + \mathbf{b}_1) \, \mathbf{W}_2 + \mathbf{b}_2.
   $$

Эти блоки повторяются несколько раз, после чего выход используется для решения задачи классификации, либо адаптируется под задачи сегментации, детекции и др.

Такой подход позволяет учитывать дальние зависимости между различными частями изображения, чего сложно добиться свертками, хотя часть эти подходы комбинируют.

Кроме того, полученное представления изображение (вектор в латентном пространстве признаков) может быть использовано для получения мультимодальных моделей.

<img src="https://viso.ai/wp-content/uploads/2021/09/vision-transformer-vit.png" alt="Описание изображения" width="600">

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm


class ViTForMNIST(nn.Module):
    def __init__(self, image_size=28, patch_size=7, in_channels=1,
                 num_classes=10, embed_dim=64, depth=6, num_heads=4,
                 dropout=0.1):
        super(ViTForMNIST, self).__init__()

        self.num_patches = (image_size // patch_size) ** 2
        self.patch_size = patch_size

        self.patch_embed = nn.Conv2d(in_channels, embed_dim,
                                     kernel_size=patch_size, stride=patch_size)
        
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, 
                                                  embed_dim))
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, nhead=num_heads, dropout=dropout,
            batch_first=True
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )
        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.patch_embed.weight)
        if self.patch_embed.bias is not None:
            nn.init.zeros_(self.patch_embed.bias)
        
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)

        self.patch_embeddings = x.detach().cpu()

        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        x = x + self.pos_embed
        x = self.dropout(x)

        x = self.encoder(x)
        self.encoder_output = x.detach().cpu()

        x_cls = x[:, 0]
        logits = self.mlp_head(x_cls)
        return logits


def train_vit(model, device, train_loader, optimizer, criterion, epoch):
    """
    Обучение модели ViT на одной эпохе.
    """
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in tqdm(enumerate(train_loader), 
                                          total=len(train_loader),
                                          desc=f'Epoch {epoch}'):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * data.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}. Loss: {epoch_loss:.4f}")
    return epoch_loss


def test_vit(model, device, test_loader, criterion):
    """
    Тестирование модели ViT на тестовой выборке.
    """
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, "
          f"Accuracy: {correct}/{len(test_loader.dataset)} "
          f"({accuracy:.2f}%)")
    return test_loss, accuracy


def visualize_vit_activations(model, device, image):
    """
    Визуализация активаций модели ViT для одного изображения.
    """
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        _ = model(image)
    patch_emb = model.patch_embeddings
    patch_emb_avg = patch_emb.mean(dim=2).squeeze(0).numpy()
    grid_patch = patch_emb_avg.reshape(int(np.sqrt(model.num_patches)),
                                       int(np.sqrt(model.num_patches)))

    encoder_out = model.encoder_output[:, 1:, :]
    encoder_out_avg = encoder_out.mean(dim=2).squeeze(0).numpy()
    grid_encoder = encoder_out_avg.reshape(int(np.sqrt(model.num_patches)),
                                           int(np.sqrt(model.num_patches)))

    fig, axes = plt.subplots(1, 2, figsize=(8, 3))
    axes[0].imshow(grid_patch, cmap='viridis')
    axes[0].set_title('Усреднённые Patch Embeddings')
    axes[0].axis('off')
    axes[1].imshow(grid_encoder, cmap='viridis')
    axes[1].set_title('Усреднённый Encoder Output')
    axes[1].axis('off')
    plt.tight_layout()
    plt.show()


batch_size = 64
test_batch_size = 1000
epochs = 3
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, 
                               transform=transform)
subset_train = np.arange(10000)
train_dataset = Subset(train_dataset, subset_train)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, 
                              transform=transform)
subset_test = np.arange(1000)
test_dataset = Subset(test_dataset, subset_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, 
                         shuffle=False)

model = ViTForMNIST().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(1, epochs + 1):
    t_loss = train_vit(model, device, train_loader, 
                       optimizer, criterion, epoch)
    train_losses.append(t_loss)
    t_loss, acc = test_vit(model, device, test_loader, criterion)
    test_losses.append(t_loss)
    test_accuracies.append(acc)

epochs_range = np.arange(1, epochs + 1)
fig, ax1 = plt.subplots(figsize=(8, 4))
ax1.plot(epochs_range, train_losses, 'o-', label='Train Loss')
ax1.plot(epochs_range, test_losses, 's-', label='Test Loss')
ax1.set_xlabel('Эпоха')
ax1.set_ylabel('Loss')
ax1.set_title('Сходимость и метрика распознавания ViT')
ax1.legend(loc='upper left')
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(epochs_range, test_accuracies, 'd--', 
         color='red', label='Test Accuracy')
ax2.set_ylabel('Accuracy (%)')
ax2.legend(loc='upper right')

plt.tight_layout()
plt.show()

sample_image, sample_label = test_dataset[0]
sample_image = sample_image.unsqueeze(0)
visualize_vit_activations(model, device, sample_image)

## Применение готовых архитектур 

Построение модели для каждой конкретной задачи затратно по времени, а также требует больших объемов данных и длительного процесса обучения.

Тут проявляется себя свойство обобщения у нейронный сетей - если решать какую-либо общую задачу, то от нее можно перейти к частной. Таким образом готовые модели, веса которых зачастую доступны открыто, обученные на огромных датасетах, могут быть легко адаптированы для конкретных задач, поскольку в них уже хорошо обучены слои извлекающие признаки. Такой подход называется transfer learning.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm


def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(
            tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}: Train Loss: {epoch_loss:.4f}")
    return epoch_loss


def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100.0 * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return test_loss, accuracy


batch_size = 64
test_batch_size = 1000
epochs = 3
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

transform = transforms.Compose([transforms.Resize(224),
                                transforms.Grayscale(num_output_channels=3),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

train_dataset = datasets.MNIST(root="./data", train=True, download=True,
                                transform=transform)
subset_train = np.arange(10000)
train_dataset = Subset(train_dataset, subset_train)

test_dataset = datasets.MNIST(root="./data", train=False, download=True,
                                transform=transform)
subset_test = np.arange(1000)
test_dataset = Subset(test_dataset, subset_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                          shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, 
                         shuffle=False)

model = torchvision.models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)

model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=learning_rate)

train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(1, epochs + 1):
    t_loss = train(model, device, train_loader, criterion, optimizer, epoch)
    train_losses.append(t_loss)
    t_loss, acc = test(model, device, test_loader, criterion)
    test_losses.append(t_loss)
    test_accuracies.append(acc)

epochs_range = np.arange(1, epochs + 1)
plt.figure(figsize=(8, 6))
plt.plot(epochs_range, train_losses, "o-", label="Train Loss")
plt.plot(epochs_range, test_losses, "s-", label="Test Loss")
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Сходимость ResNet18 на MNIST")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(epochs_range, test_accuracies, "d--", 
         color="red", label="Test Accuracy")
plt.xlabel("Эпоха")
plt.ylabel("Accuracy (%)")
plt.title("Точность ResNet18 на MNIST")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm


def train(model, device, train_loader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(
            tqdm(train_loader, desc=f"Epoch {epoch}", leave=False)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * data.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}: Train Loss: {epoch_loss:.4f}")
    return epoch_loss


def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item() * data.size(0)
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100.0 * correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return test_loss, accuracy


batch_size = 64
test_batch_size = 1000
epochs = 3
learning_rate = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Используем устройство:", device)

transform = transforms.Compose([transforms.Resize(224),
                                transforms.Grayscale(num_output_channels=3),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])])

train_dataset = datasets.MNIST(root="./data", train=True, download=True,
                                transform=transform)
subset_train = np.arange(10000)
train_dataset = Subset(train_dataset, subset_train)

test_dataset = datasets.MNIST(root="./data", train=False, download=True,
                                transform=transform)
subset_test = np.arange(1000)
test_dataset = Subset(test_dataset, subset_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)

model = torchvision.models.vit_b_16(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

in_features = model.heads[-1].in_features
model.heads[-1] = nn.Linear(in_features, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.heads[-1].parameters(), lr=learning_rate)

train_losses = []
test_losses = []
test_accuracies = []

for epoch in range(1, epochs + 1):
    t_loss = train(model, device, train_loader, criterion, optimizer, epoch)
    train_losses.append(t_loss)
    t_loss, acc = test(model, device, test_loader, criterion)
    test_losses.append(t_loss)
    test_accuracies.append(acc)

epochs_range = np.arange(1, epochs + 1)
plt.figure(figsize=(8, 6))
plt.plot(epochs_range, train_losses, "o-", label="Train Loss")
plt.plot(epochs_range, test_losses, "s-", label="Test Loss")
plt.xlabel("Эпоха")
plt.ylabel("Loss")
plt.title("Сходимость ViT на MNIST")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
plt.plot(epochs_range, test_accuracies, "d--", 
         color="red", label="Test Accuracy")
plt.xlabel("Эпоха")
plt.ylabel("Accuracy (%)")
plt.title("Точность ViT на MNIST")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()